# Cipher Decription
* This project is aimed at producing a decipher that discovers the correct mapping &lt;code_letter&gt;:&lt;real_letter&gt; from secret messages.
* The solution is based on the concept of Genetic Algorithms. Each decipher is basically a dictionary that represents a hypothetical mapping of letters.
* Each offspring will inherit some of the mappings of their parent's dictionary, but with a slight modification/mutation.

In [18]:
from re import sub
from typing import List

def load_text(filename:str)->List[str]:
    '''
        Reads the .txt file.
        
        Parameter
        ---------
        `filename`: str
            The name of the poems file.
            
        Returns
        -------
        A list containing each strophe's content.
    '''
    with open(f'data/{filename}', 'r') as f:
        parag_delim = '\n\n'
        text = sub('(\ufeff|\n)?CHAPTER \d+\.[\w\'`,;\. \-]*', '', f.read()).split(parag_delim)
        return text
    
corpus_model = load_text('moby_dick.txt')
corpus_model[:3]

<>:19: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_194566/595657345.py:19: SyntaxWarning: invalid escape sequence '\d'
  text = sub('(\ufeff|\n)?CHAPTER \d+\.[\w\'`,;\. \-]*', '', f.read()).split(parag_delim)


['',
 'Call me Ishmael. Some years ago—never mind how long precisely—having\nlittle or no money in my purse, and nothing particular to interest me\non shore, I thought I would sail about a little and see the watery part\nof the world. It is a way I have of driving off the spleen and\nregulating the circulation. Whenever I find myself growing grim about\nthe mouth; whenever it is a damp, drizzly November in my soul; whenever\nI find myself involuntarily pausing before coffin warehouses, and\nbringing up the rear of every funeral I meet; and especially whenever\nmy hypos get such an upper hand of me, that it requires a strong moral\nprinciple to prevent me from deliberately stepping into the street, and\nmethodically knocking people’s hats off—then, I account it high time to\nget to sea as soon as I can. This is my substitute for pistol and ball.\nWith a philosophical flourish Cato throws himself upon his sword; I\nquietly take to the ship. There is nothing surprising in this. If they\nb

In [19]:
from typing import List

def filter_empty_strings(corpus:List[str])->List[str]:
    '''
        Some of the treatments made might have generated empty strings. Thus, we need to remove them to guarantee
        the success of our model's training.
        
        Parameter
        ---------
        `corpus`: List[str]
            Our project's corpus.
        
        Returns
        -------
        Our corpus with no empty strings.
    '''
    _lambda = lambda s: len(s)>0
    corpus = list(filter(_lambda, corpus))
    return corpus

corpus_model = filter_empty_strings(corpus_model)
corpus_model[:3]

['Call me Ishmael. Some years ago—never mind how long precisely—having\nlittle or no money in my purse, and nothing particular to interest me\non shore, I thought I would sail about a little and see the watery part\nof the world. It is a way I have of driving off the spleen and\nregulating the circulation. Whenever I find myself growing grim about\nthe mouth; whenever it is a damp, drizzly November in my soul; whenever\nI find myself involuntarily pausing before coffin warehouses, and\nbringing up the rear of every funeral I meet; and especially whenever\nmy hypos get such an upper hand of me, that it requires a strong moral\nprinciple to prevent me from deliberately stepping into the street, and\nmethodically knocking people’s hats off—then, I account it high time to\nget to sea as soon as I can. This is my substitute for pistol and ball.\nWith a philosophical flourish Cato throws himself upon his sword; I\nquietly take to the ship. There is nothing surprising in this. If they\nbut kn

In [20]:
import string
from typing import List

def remove_punctuation(s:str)->str:
    '''
        Removes punctuation from a string. Also, already normalizes (lower) and removes any whitespaces
        from the string's borders.
        
        Parameter
        ---------
        s: `str`
            The provided string.
        
        Returns
        -------
        The treated string.
    '''
        
    translation_table = str.maketrans('', '', string.punctuation+ '—“”‘‘’\n')
    return s.lower().strip().translate(translation_table)

def treat(s:str)->str:
    '''
        Applies all transformations mentioned above in a text.
        
        Parameter
        ---------
        s: `str`
            The provided string.

        Returns
        -------
        The treated string.
    '''
    s = remove_punctuation(s)
    s = s.replace(' ','') # Note that I'm disregarding spaces since passwords commonly don't have it.
    return s

corpus_model = list(map(treat, corpus_model))

In [21]:
from re import fullmatch

def keep_alpha(corpus:List[str])->List[str]:
    '''
        For the project's purpose, keep only fragments  that contain letters.
        
        Parameter
        ---------
        `corpus`: List[str]
            The project's corpus.
            
        Returns
        -------
        The filtered corpus.
    '''
    _lambda = lambda s: fullmatch('[A-z]+', s)
    return list(filter(_lambda, corpus))

corpus_model = keep_alpha(corpus_model)

## Evaluator Setting
* Since comparing our keys and values to the actual mapping is unadequate for a real world application, we'll instead use a Markov Model. It is going to receive the Moby Dick's corpus and will learn the most common unigrams and bigrams of the English Language.
* Therefore, ẃe'll expect that our best decipher is going to create the sequences with higher probability, according to the model.

In [22]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from typing import List, Tuple

class MarkovModel:
    '''
       Markov Model, with Add-Epsilon Smoothing.

        Parameters
        ---------
        `corpus`: List[str]
            List with the documents to be used.
        `epsilon`: float
            Smoothing degree of the probabilities.
        `name`: str
            A name for your model.
            
        Methods
        ------
        `fit`: Generates the model's A and pi.
        `predict_log_proba`: Estimates the probability's log of a given sequence.
        
        
        Attributes
        ----------
        `a`: `pd.DataFrame`
            The model's A matrix.
        `pi`: `pd.Series`
            The model's pi vector.
        `_vocab`: Set[str]
            A set object with all the corpus's vocabulary.
    '''
    def __init__(self, corpus:List[str], epsilon:float, name:str):
        self.corpus = corpus
        self.corpus_length = len(self.corpus)
        self.epsilon = epsilon
    
    def __vocab(self)->None:
        '''
            Extraction of all the corpus tokens.
            
            We create a set with all training tokens and another one disregarding the ones only used as first word of the strophes.
        '''
        self._vocab, self._a_vocab = [], []
        
        for doc in self.corpus:
            self._vocab += doc
            self._a_vocab+=doc[1:] # Not including the first tokens.
            
        self._vocab, self._a_vocab = set(self._vocab), set(self._a_vocab)        
    
    def __check_pi(self, token:str)->str:
        '''
            Masks a sentence's first token with '<UNKNOWN>' mark if it is not included in the training set.
            
            Parameter
            ---------
            `token`: str
                The sentence's first token under scrutiny
            
            Returns
            -------
            The treated token.
        '''
        return token if token in self.pi else '<UNKNOWN>'
    
    def __check_a(self, token1:str, token2:str)->Tuple[str]:
        '''
            When querying the model's A matrix, checks whether the provided initial and target states are present. If not,
            the tokens are masked with the flag '<UNKNOWN>'.
            
            Parameters
            ----------
            `token1`: str
                The initial state.
            `token2`: str
                The target state.
            
            Returns
            -------
            The treated tokens inside a tuple.
        '''
        token1 = token1 if token1 in self.a.index else '<UNKNOWN>'
        token2 = token2 if token2 in self.a.columns else '<UNKNOWN>'
        return token1, token2
    
    @property
    def pi(self):
        '''
            Encharged for measuring the model's pi vector.
        '''
        pi = {}
        m = self.a.shape[0]
        
        for doc in self.corpus:
            i = doc[0]
            if i not in pi.keys():
                pi[i] = 1
            else:
                pi[i]+=1
        
        pi['<UNKNOWN>'] = 0 # Defining a key for possible tokens of the test set that were unseen during training.
        pi =  (pd.Series(pi)+self.epsilon) / (self.corpus_length+self.epsilon*m)
        return pi
        
    @property
    def a(self):
        '''
            Measures the model's A matrix.
        '''
        a = {j:{} for j in self._a_vocab}
        for doc in self.corpus:
            for idx, j in enumerate(doc[1:], start=1):
                d_j = a[j]
                i = doc[idx-1]
                if i not in d_j.keys():
                    d_j[i] = 1
                else:
                    d_j[i] += 1
        a['<UNKNOWN>'] = {'<UNKNOWN>':0}
        a = pd.DataFrame(a).fillna(0)
        
        # Essential elements for our Transition Matrix's computing.
        m = a.shape[0]
        num = (a+self.epsilon)
        denom = a.sum(axis=1, skipna=True)+m*self.epsilon
        
        # Computing the matrix.
        a =  num.div(denom, axis=0) 
        return a

    def fit(self):
        '''
            Fits the algorithm to the provided corpus.
        '''
        self.__vocab()
        self.a
        self.pi
        return self
    
    def predict_log_proba(self, text:str)->float:
        '''
            Estimates the probability's log of a given sequence.
            
            Parameter
            ---------
            `text`: str
                The text whose probability needs to be computed.
            
            Returns
            -------
            The sequence's log probability.
        '''
        #text = word_tokenize(text.lower())
        proba_pi = np.log(self.pi[self.__check_pi(text[0])])
        proba_a = np.log([self.a.loc[self.__check_a(text[i-1], text[i])] for i in range(1, len(text))]) 
        return proba_pi + np.sum(proba_a)                                                   
    
    def predict_proba(self, text:str)->float:
        '''
            Estimates the probability of a given sequence.
            
            *Note:* There is a risk of the output to be 0 for long sequences.
            
            Parameter
            ---------
            `text`: str
                The text whose probability needs to be computed.
            
            Returns
            -------
            The sequence's probability.
        '''
        return np.exp(self.predict_log_proba(text))

In [23]:
# Setting our project's evaluator. 
model = MarkovModel(corpus_model, 1, 'hi').fit()

## Loading the Ciphers Data
* Our dataset is going to be comprised of a small sample of alphabetic passwords whose characters will be mapped to other ones using a dictionary.

In [24]:
# The passwords below were generated by GPT-3.5
passwords = [
    'lemonadebasket',
    'butterflywhisper',
    'moonlightdream',
    'starrynightfall',
    'oceanwavesdance',
    'mountainpeakview',
    'riversidebreeze',
    'forestpathwalk',
    'sunnydaymeadow',
    'autumnleaffall',
    'winterfrostbite',
    'springblossomjoy',
    'summerheatwave',
    'fireflyglimmer',
    'hummingbirdbuzz',
    'dragonflydance',
    'ladybuggarden',
    'butterflyflutter',
    'raindropserenade',
    'snowflakedrift',
    'seashelltreasure',
    'stardustwhisper',
    'sandcastlemagic',
    'bonfirecrackle',
    'thunderstormrumble',
    'lightningboltflash',
    'waterfallcascade',
    'rainbowcolorsplash',
    'sunsethorizon',
    'sunrisebeauty'
]

In [25]:
from string import ascii_lowercase
from random import sample, seed
from typing import Dict

def make_mapping(random_seed:int)->Dict[str, str]:
    '''
        Creates a random mapping for each of the alphabet's letters.
        
        Parameter
        ---------
        `random_seed`: int
            The seed that is going to base the mapping generation.
            
        Returns
        -------
        A dictionary with the mapping <actual_letter>:<mask_letter>
    '''
    seed(random_seed)
    return dict(zip(*(ascii_lowercase, sample(ascii_lowercase, k=26))))


# Instantiating the mapping to be used to occult the passwords.
cipher_map = make_mapping(42)
cipher_map

{'a': 'u',
 'b': 'd',
 'c': 'a',
 'd': 'i',
 'e': 'h',
 'f': 'v',
 'g': 'e',
 'h': 'y',
 'i': 'r',
 'j': 'c',
 'k': 'n',
 'l': 'x',
 'm': 'o',
 'n': 'b',
 'o': 's',
 'p': 'l',
 'q': 'w',
 'r': 'p',
 's': 'k',
 't': 'f',
 'u': 'g',
 'v': 't',
 'w': 'z',
 'x': 'j',
 'y': 'm',
 'z': 'q'}

In [26]:
# Now, we can use such mapping to encipher a password.
def encipher(s:str, mapping:Dict[str, str])->str:
    '''
        Enciphers a given string based on a mapping.
        
        Parameters
        ----------
        `s`: str
            The string.
        `mapping`: Dict[str, str]
            The provided mapping.
            
        Returns
        -------
        The encrypted string.
    '''
    output = ''
    for letter in s:
        output += mapping[letter]
    return output

# An example of ciphering the first password.
passwords[0], encipher(passwords[0], cipher_map)

('lemonadebasket', 'xhosbuihduknhf')

In [27]:
# Now, enciphering the passwords.
ciphered_passwords = list(map(lambda s: encipher(s, cipher_map), passwords))
ciphered_passwords

['xhosbuihduknhf',
 'dgffhpvxmzyrklhp',
 'ossbxreyfiphuo',
 'kfuppmbreyfvuxx',
 'sahubzuthkiubah',
 'osgbfurblhuntrhz',
 'prthpkrihdphhqh',
 'vsphkflufyzuxn',
 'kgbbmiumohuisz',
 'ugfgobxhuvvuxx',
 'zrbfhpvpskfdrfh',
 'klprbedxskksocsm',
 'kgoohpyhufzuth',
 'vrphvxmexroohp',
 'ygoorbedrpidgqq',
 'ipuesbvxmiubah',
 'xuimdgeeupihb',
 'dgffhpvxmvxgffhp',
 'purbipslkhphbuih',
 'kbszvxunhiprvf',
 'khukyhxxfphukgph',
 'kfupigkfzyrklhp',
 'kubiaukfxhouera',
 'dsbvrphapuanxh',
 'fygbihpkfspopgodxh',
 'xreyfbrbedsxfvxuky',
 'zufhpvuxxaukauih',
 'purbdszasxspklxuky',
 'kgbkhfysprqsb',
 'kgbprkhdhugfm']

## Genetic Algorithm Creation
* With all our passwords properly encrypted, we are free to program our algorithm.
* It will basically consist in another dictionary with some other hypothetical letter to letter mapping.
* At each iteration, we'll enforce our current cipher to produce an offspring. Each descendant will carry most of its parent's mapping with a slight change - or "mutation".
* We'll continue this reproduction cycle with the most successful descendant of the offspring, until no child presents better performances than its parent or we reach the maximum amount of iterations!

In [69]:
mapping = make_mapping(10)

class Cipher:
    def __init__(self, mapping:Dict[str, str]):
        self.mapping = mapping
        
    def decode(self, s:str)->str:
        return encipher(s, self.mapping)

    def mutate(self, eta:float):
        eta = int(eta*26)
        keys = list(self.mapping.keys())
        keys_mutate = np.random.choice(keys, size=eta, replace=False)
        new_values = np.random.choice([self.mapping[k] for k in keys_mutate], size=eta, replace=False)
        for k,v in zip(keys_mutate, new_values):
            self.mapping[k]=v

In [70]:
from typing import List

def offspring(cipher:Cipher, n:int, eta:float)->List[Cipher]:
    return [cipher.mutate(eta) for _ in range(n)]

offspring(Cipher(mapping), 10, .5)

[None, None, None, None, None, None, None, None, None, None]

<p style='color:red'> Criei a função de mutação; Criar o mecanismo de reprodução dos Ciphers</p>